In [ ]:
import wordcloud as wc
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import spacy
nlp = spacy.load('nl_core_news_sm')

In [ ]:
df = pd.read_csv('data/BGT-Bijbel.csv', sep='|')

In [ ]:
books = df['book'].unique()
books

In [ ]:
def get_counts(book):
    text = ' '.join(df[df['book'] == book]['text'])
    doc = nlp(text)
    # for token in doc:
    #     print(token.text, token.pos_, token.dep_)
    poss = ('NOUN', 'PROPN', 'ADJ', 'VERB')
    tokens = (token.text for token in doc if token.pos_ in poss)
    counts = Counter(tokens)
    return counts

def get_wordcloud(counts):
    wordcloud = wc.WordCloud(
        width=600, 
        height=600, 
        colormap='jet', 
        background_color='white'
    ).generate_from_frequencies(counts)
    
    return wordcloud

In [ ]:
all_counts = Counter()

for book in books:
    counts = get_counts(book)
    wordcloud = get_wordcloud(counts)
    
    all_counts.update(counts)

    fig, ax = plt.subplots(figsize=(6,6))

    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_axis_off()
    ax.set_title(book)
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

wordcloud = get_wordcloud(all_counts)
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()
ax.set_title('Bijbel')
plt.show()